# Recommendation Systems - Lab

## Introduction

Now that you've gotten an introduction to collaborative filtering and recommendation systems, it's time to put your skills to test and attempt to build a recommendation system for a real world dataset! For this exercise, you'll be using a dataset regarding the book reviews on the Amazon marketplace. While the previous lesson focused on user-based recommendation systems, you'll apply a parallel process for an item-based recommendation system to recommend similar books at the bottom of the product page.

## Objectives

You will be able to:
* Implement a recommendation system on a real world dataset

## Load the Dataset

In [54]:
#Your code here

import pandas as pd
df = pd.read_csv('books_data.edgelist', names =  ['source', 'target', 'weight'], delimiter = ' ')
df.head()


,source,target,weight
0,0827229534,0804215715,0.7
1,0827229534,156101074X,0.5
2,0827229534,0687023955,0.8
3,0827229534,0687074231,0.8
4,0827229534,082721619X,0.7


## Load the MetaData

In [55]:
#Your code here

meta = pd.read_csv('books_meta.txt', sep = '\t')
meta.head()

,Id,ASIN,Title,Categories,Group,SalesRank,TotalReviews,AvgRating,DegreeCentrality,ClusteringCoeff
0,1,0827229534,Patterns of Preaching: A Sermon Sampler,clergi sermon subject religion preach spiritu ...,Book,396585,2,5.0,8,0.80
1,2,0738700797,Candlemas: Feast of Flames,subject witchcraft earth religion spiritu base...,Book,168596,12,4.5,9,0.85
2,3,0486287785,World War II Allied Fighter Planes Trading Cards,general hobbi subject craft home garden book,Book,1270652,1,5.0,0,0.00
3,4,0842328327,Life Application Bible Commentary: 1 and 2 Tim...,spiritu translat commentari christian book gui...,Book,631289,1,4.0,6,0.79
4,5,1577943082,Prayers That Avail Much for Business: Executive,subject religion spiritu busi christian live w...,Book,455160,0,0.0,4,1.00


## Select Books to Test Your Recommender On

Select a small subset of books that you are interested in generating recommendations for. 

In [56]:
#Your code here

GOT = meta[meta.Title.str.contains('Thrones')]
GOT.head()

,Id,ASIN,Title,Categories,Group,SalesRank,TotalReviews,AvgRating,DegreeCentrality,ClusteringCoeff
59750,83450,0553103547,"A Game of Thrones (A Song of Ice and Fire, Boo...",general subject martin author epic z seri fant...,Book,16330,1191,4.5,4,0.67
130560,182190,1572701293,"Thrones, Dominations",literatur seri tape book format fiction genera...,Book,395606,61,3.5,4,0.90
183820,256164,0553573403,"A Game of Thrones (A Song of Ice and Fire, Boo...",general subject martin author epic z seri fant...,Book,969,1196,4.5,7,0.48
261763,362549,0553381687,"A Game of Thrones (A Song of Ice and Fire, Boo...",general subject martin author epic z seri fant...,Book,11463,1196,4.5,4,0.67
331188,457079,0312968302,"Thrones, Dominations (A Lord Wimsey Mystery)",general subject british author sayer dorothi l...,Book,68918,61,3.5,7,0.81


## Generate Recommendations for a Few Books of Choice

The 'books_data.edgelist' has conveniently already calculated the distance between items for you. Given this preprocessed and data, it's time to employ collaborative filtering to generate recommendations! Generate the top 10 recommendations for each book in the subset you chose. Be sure to print the book name that you are generating recommendations for as well as the name of the books being recommended.

In [57]:
GOT.iloc[1].ASIN

'1572701293'

In [58]:
#Your code here

recommendations = {}
#source  = ASIN, so create corresponding dictionary for titles and source
id_dict = dict(zip(meta.ASIN, meta.Title))
id_dict

{'0827229534': 'Patterns of Preaching: A Sermon Sampler',
 '0738700797': 'Candlemas: Feast of Flames',
 '0486287785': 'World War II Allied Fighter Planes Trading Cards',
 '0842328327': 'Life Application Bible Commentary: 1 and 2 Timothy and Titus',
 '1577943082': 'Prayers That Avail Much for Business: Executive',
 '0486220125': 'How the Other Half Lives: Studies Among the Tenements of New York',
 '0231118597': 'Losing Matt Shepard',
 '1859677800': 'Making Bread: The Taste of Traditional Home-Baking',
 '0375709363': 'The Edward Said Reader',
 '0871318237': 'Resetting the Clock : Five Anti-Aging Hormones That Improve and Extend Life',
 '1590770218': 'Fantastic Food with Splenda : 160 Great Recipes for Meals Low in Sugar, Carbohydrates, Fat, and Calories',
 '0313230269': 'Clockwork Worlds : Mechanized Environments in SF (Contributions to the Study of Science Fiction and Fantasy)',
 '1559362022': 'Wake Up and Smell the Coffee',
 '0195110382': 'War at Sea: A Naval History of World War II',


In [64]:


for index in list(GOT.index):
    book_id = meta.iloc[index].ASIN
    book_title = id_dict[book_id]
    top_ten_similar = df[(df.source == book_id) | (df.target == book_id)].sort_values(by = 'weight', ascending = False).head(10)
    top_ten_similar['source_name'] = top_ten_similar['source'].map(id_dict)
    top_ten_similar['target_name'] = top_ten_similar['target'].map(id_dict)
    recs = []
    for i in top_ten_similar.index:
        if top_ten_similar.source[i] == book_id:
            recs.append((top_ten_similar.target_name[i], top_ten_similar.weight[i]))
        else:
            recs.append((top_ten_similar.source_name[i], top_ten_similar.weight[i]))
    recommendations[book_title] = recs
    print('Recs for {}'.format(book_title))

    for rec in recommendations[book_title]:
        print(rec)
    print()

Recs for A Game of Thrones (A Song of Ice and Fire, Book 1)
('A Clash of Kings (A Song of Ice and Fire, Book 2)', 1.0)
('A Feast for Crows (A Song of Ice and Fire, Book 4)', 0.92)
('A Storm of Swords (A Song of Ice and Fire, Book 3)', 0.85)
("Assassin's Apprentice (The Farseer Trilogy, Book 1)", 0.56)

Recs for Thrones, Dominations
('Have His Carcase', 0.59)
('The Nine Tailors', 0.58)
('Strong Poison', 0.55)
("Busman's Honeymoon", 0.55)

Recs for A Game of Thrones (A Song of Ice and Fire, Book 1)
('A Storm of Swords : Book Three of A Song of Ice and Fire (A Song of Ice and Fire, Book 3)', 1.0)
('A Storm of Swords (A Song of Ice and Fire, Book 3)', 1.0)
('A Clash of Kings (A Song of Ice and Fire, Book 2)', 1.0)
('A Feast for Crows (A Song of Ice and Fire, Book 4)', 0.92)
('A Storm of Swords (A Song of Ice and Fire, Book 3)', 0.85)
("Assassin's Apprentice (The Farseer Trilogy, Book 1)", 0.56)
('The Fourth Tower of Inverness', 0.24)

Recs for A Game of Thrones (A Song of Ice and Fire, Boo

## Summary

Well done! In this lab, you effectively created a recommendation system for a real world dataset!